In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1735197544.698679   64466 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735197544.701971   64466 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [ ]:
f.predictions('smr')

SMR 5m Interval Timestamp: 2024-12-26 03:11:40


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.212903,0.21562,0.196788,0.134421,0.238926
prob_up,0.298161,0.296696,0.344178,0.375,0.0
prob_static,0.477851,0.542148,0.450395,0.5,1.0
prob_down,0.223988,0.161156,0.205426,0.125,0.0
precision,0.437788,0.439728,0.426277,0.381729,0.456376
recall,0.611588,0.555794,0.626609,0.502146,0.437768
f1,0.510295,0.490995,0.507385,0.433735,0.446878
support,"[463.0, 466.0, 461.0]","[463.0, 466.0, 461.0]","[463.0, 466.0, 461.0]","[463.0, 466.0, 461.0]","[463.0, 466.0, 461.0]"


SMR 15m Interval Timestamp: 2024-12-26 03:12:14


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.267059,0.29125,0.226744,0.171144,0.146667
prob_up,0.158093,0.058301,0.181984,0.25,0.0
prob_static,0.751805,0.889934,0.646837,0.625,1.0
prob_down,0.090102,0.051766,0.171179,0.125,0.0
precision,0.476471,0.49375,0.447674,0.40796,0.390476
recall,0.522581,0.509677,0.496774,0.529032,0.264516
f1,0.498462,0.501587,0.470948,0.460674,0.315385
support,"[155.0, 155.0, 155.0]","[155.0, 155.0, 155.0]","[155.0, 155.0, 155.0]","[155.0, 155.0, 155.0]","[155.0, 155.0, 155.0]"


SMR 1h Interval Timestamp: 2024-12-26 03:12:30


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,up,up
kelly_1:2.5,0.24281,0.24744,0.24688,0.069847,0.066667
prob_up,0.281737,0.271902,0.259849,0.375,1.0
prob_static,0.395093,0.367262,0.375892,0.25,0.0
prob_down,0.32317,0.360836,0.364259,0.375,0.0
precision,0.45915,0.462457,0.462057,0.335605,0.333333
recall,0.52037,0.501852,0.507407,0.364815,1.0
f1,0.487847,0.48135,0.483672,0.349601,0.5
support,"[540.0, 540.0, 540.0]","[540.0, 540.0, 540.0]","[540.0, 540.0, 540.0]","[540.0, 540.0, 540.0]","[540.0, 540.0, 540.0]"


SMR 1d Interval Timestamp: 2024-12-26 03:13:20


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,down,up
kelly_1:2.5,0.095833,0.075472,0.178261,0.033333,0.080292
prob_up,0.798231,0.892597,0.663461,0.375,1.0
prob_static,0.025098,0.020443,0.092461,0.0,0.0
prob_down,0.17667,0.08696,0.244077,0.625,0.0
precision,0.354167,0.339623,0.413043,0.309524,0.343066
recall,0.361702,0.382979,0.404255,0.276596,1.0
f1,0.357895,0.36,0.408602,0.292135,0.51087
support,"[47.0, 43.0, 47.0]","[47.0, 43.0, 47.0]","[47.0, 43.0, 47.0]","[47.0, 43.0, 47.0]","[47.0, 43.0, 47.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')